In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [6]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy
import pandas as pd

def backtesting(data, plot_flag=False):
    balance = 100    
    def pivot(df_campo):
        indi=pd.Series(df_campo)
        return indi.to_numpy()
    class Fenix(Strategy):
        def init(self):
            super().init()
            self.pivot_high = self.I(pivot,self.data.pivot_high)
            self.pivot_low = self.I(pivot,self.data.pivot_low)
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def detect_pivot_points(df):
    # Función para detectar los puntos pivote High y Low
    # Parámetros de longitud para los puntos pivote High y Low
    leftLenH = 10
    rightLenH = 10
    leftLenL = 10
    rightLenL = 10
    df['pivot_high'] = np.NaN
    df['pivot_low'] = np.NaN
    df['row_number'] = (range(len(df)))
    df.set_index('row_number', inplace=True)
    ## PIVOTS SUPERIORES
    for i in range(leftLenH, len(df) - rightLenH):
        if (
            df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
            and df['High'].iloc[i] > df['High'].iloc[i - 1]
            and df['High'].iloc[i] > df['High'].iloc[i + 1]
            ):
            df.at[i, 'pivot_high'] = df['High'].iloc[i]
    ## PIVOTS INFERIORES
    for i in range(leftLenL, len(df) - rightLenL):
        if (
            df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
            and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
            and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
            ):
            df.at[i, 'pivot_low'] = df['Low'].iloc[i]
    ## relleno
    for i in range(0, len(df) - 1):
        if np.isnan(df['pivot_low'].iloc[i]):
            df.at[i, 'pivot_low'] = df['pivot_low'].iloc[i - 1]
        if np.isnan(df['pivot_high'].iloc[i]):
            df.at[i, 'pivot_high'] = df['pivot_high'].iloc[i - 1]

    df['timestamp']=df['Open Time']
    df.set_index('timestamp', inplace=True)

##########################################################################################

def estrategia_real(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'15m')      
        detect_pivot_points(data)  
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################


symbol='ATOMUSDT'
data = estrategia_real(symbol)
backtesting(data, plot_flag=True)
#for index, values in df.iterrows():
#    if values.PivotLow ==1 or values.PivotHigh ==1:
#        print(f"{values['Open Time']} - {values.PivotLow} - {values.PivotHigh}")

data.tail(60)     

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,pivot_high,pivot_low,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-09-16 07:30:00,2023-09-16 07:30:00,7.080,7.084,7.065,7.068,39673.20,7.063759,7.000373,6.827844,0.029605,7.127,6.778,0,NaN,NaN,False,238
2023-09-16 07:45:00,2023-09-16 07:45:00,7.069,7.092,7.065,7.081,47761.62,7.065401,7.003535,6.830363,0.029419,7.127,6.778,0,NaN,NaN,False,240
2023-09-16 08:00:00,2023-09-16 08:00:00,7.082,7.099,7.067,7.068,67951.20,7.065648,7.006063,6.832728,0.029603,7.127,6.778,0,NaN,NaN,False,238
2023-09-16 08:15:00,2023-09-16 08:15:00,7.067,7.098,7.064,7.088,78411.07,7.067777,7.009276,6.835268,0.029917,7.127,6.778,0,NaN,NaN,False,236
2023-09-16 08:30:00,2023-09-16 08:30:00,7.088,7.105,7.088,7.091,41522.72,7.069989,7.012481,6.837812,0.028995,7.127,6.778,0,NaN,NaN,False,244
2023-09-16 08:45:00,2023-09-16 08:45:00,7.090,7.096,7.070,7.078,51710.53,7.070752,7.015051,6.840202,0.028781,7.127,6.778,0,NaN,NaN,False,245
2023-09-16 09:00:00,2023-09-16 09:00:00,7.078,7.119,7.077,7.116,103377.36,7.075061,7.019009,6.842946,0.029725,7.127,6.778,0,NaN,NaN,False,239
2023-09-16 09:15:00,2023-09-16 09:15:00,7.116,7.128,7.110,7.126,62335.30,7.079912,7.023205,6.845763,0.028888,7.127,6.778,0,NaN,NaN,False,246
2023-09-16 09:30:00,2023-09-16 09:30:00,7.125,7.139,7.113,7.126,115654.04,7.084302,7.027236,6.848551,0.028681,7.127,6.778,0,NaN,NaN,False,248
2023-09-16 09:45:00,2023-09-16 09:45:00,7.126,7.136,7.095,7.100,115038.14,7.085797,7.030090,6.851053,0.029561,7.127,6.778,0,NaN,NaN,False,240
